In [12]:
import pickle
from qiskit.quantum_info import Clifford
from qiskit import QuantumCircuit, qasm2
from qiskit.circuit.library import *
from generate_cayley_graph import Node
from testing import *
from shortest_path import dijkstra
import pandas as pd
import time

In [2]:
def optimization_subroutine_2_qubits(qc):
    
    

SyntaxError: incomplete input (1892306611.py, line 3)

In [3]:
generator_gates = [HGate(),SGate(),CXGate()] #Entire Clifford Set

n_qubits = 5
n_gates = 10


random_circ = generate_random_circuit(generator_gates,n_gates, n_qubits)

In [8]:
with open(f"cayley_graph_2_qubits.pkl", "rb") as f:
    nodes = pickle.load(f)
    
    n_qubits = 2
    n_start_gates = 4

    identity = Node(Clifford.from_circuit(QuantumCircuit(n_qubits)))

    target = Node(get_random_clifford(n_qubits, n_start_gates))

    shortest_circuit, length = dijkstra(nodes, identity.key, target.key)
    print(f"Shortest circuit: {shortest_circuit}")
    print(f"Reduced from {n_start_gates} to {length}")

Start with circuit: ['h1', 'cx01', 'h1', 'h1']
Shortest circuit: ['H1', 'CX01']
Reduced from 4 to 2


In [55]:
Dat = pd.DataFrame(columns = ['Original Circuit',
                              'Original Circuit QASM',
                              'Qubit Count',
                              'Original Gate Count',
                              'Original Depth',
                              'H_0',
                              'S_0',
                              'CX_0',
                              'Optimized Circuit',
                              'Optimized Circuit QASM',
                              'Optimized Gate Count',
                              'Optimized Depth',
                              'H_Opt',
                              'S_Opt',
                              'CX_Opt',
                              'Time',
                              '% Gate Count Reduction',
                              '% Depth Reduction',
                              '% H Reduction',
                              '% S Reduction',
                              '% CX Reduction'
                              ])



In [56]:
GateCountRange = range(5,51) #so as to include 50
QubitCountRange = range(2,3) #so as to include 2
R = 20 #Number of circuits for given gate and qubit count
gset = [HGate(),CXGate(),SGate()]

def opt_subroutine(qc):

    with open(f"cayley_graph_2_qubits.pkl", "rb") as f:
        nodes = pickle.load(f)

        n_qubits = qc.num_qubits
    
    
        identity = Node(Clifford.from_circuit(QuantumCircuit(n_qubits)))
    
        target = Node(Clifford(qc))
    
        result = dijkstra(nodes, identity.key, target.key)

        if result is None:
            qc_opt = None
            
        else:
            path, dist = result

            qc_opt = QuantumCircuit(n_qubits)
            for gate in path:
                if gate.startswith("H"):
                    pos = int(gate[1])
                    qc_opt.h(pos)
                elif gate.startswith("S"):
                    pos = int(gate[1])
                    qc_opt.s(pos)
                elif gate.startswith("CX"):
                    ctrl, tgt = int(gate[2]), int(gate[3])
                    qc_opt.cx(ctrl,tgt)

        return qc_opt

    

def opt_subroutine_with_time(qc): #This takes in the optimization subroutine and it also computes the time within the optimization process.
    start_time = time.time()
    qc_opt = opt_subroutine(qc)
    t = (time.time() - start_time)
    return qc_opt,t


exec_count=0
for GateCount in GateCountRange:
    for QubitCount in QubitCountRange:
        for _ in range(R):
            qc = generate_random_circuit(gset, GateCount, QubitCount)
            qc_qasm = qasm2.dumps(qc)
            qc_depth = qc.depth()
            qc_gatelist = [i.operation.name for i in qc.data]
            qc_countsdict = {'h':qc_gatelist.count('h'),'s':qc_gatelist.count('s'),'cx':qc_gatelist.count('cx')}
            

            qc_opt,t = opt_subroutine_with_time(qc) 

            qc_opt_qasm = qasm2.dumps(qc_opt)
            qc_opt_gc = len(qc_opt.data)
            qc_opt_depth = qc_opt.depth()
            qc_opt_gatelist = [i.operation.name for i in qc_opt.data]
            qc_opt_countsdict = {'h':qc_opt_gatelist.count('h'),'s':qc_gatelist.count('s'),'cx':qc_opt_gatelist.count('cx')}

            DataRow = {'Original Circuit':qc,
                              'Original Circuit QASM':qc_qasm,
                              'Qubit Count':QubitCount,
                              'Original Gate Count':GateCount,
                              'Original Depth':qc_depth,
                              'H_0': qc_countsdict['h'],
                              'S_0': qc_countsdict['s'],
                              'CX_0': qc_countsdict['cx'],
                              'Optimized Circuit':qc_opt,
                              'Optimized Circuit QASM':qc_opt_qasm,
                              'Optimized Gate Count':qc_opt_gc,
                              'Optimized Depth':qc_opt_depth,
                              'H_Opt':qc_opt_countsdict['h'],
                              'S_Opt':qc_opt_countsdict['s'],
                              'CX_Opt':qc_opt_countsdict['cx'],
                              'Time':t,
                       '% Gate Count Reduction':(GateCount - qc_opt_gc)*100/GateCount,
                       '% Depth Reduction':(qc_depth - qc_opt_depth)*100/qc_depth,
                              '% H Reduction': 0 if qc_countsdict['h'] == 0 else (qc_countsdict['h'] - qc_opt_countsdict['h'])*100/qc_countsdict['h'],
                              '% S Reduction': 0 if qc_countsdict['s'] == 0 else (qc_countsdict['s'] - qc_opt_countsdict['s'])*100/qc_countsdict['s'],
                              '% CX Reduction': 0 if qc_countsdict['cx'] == 0 else (qc_countsdict['cx'] - qc_opt_countsdict['cx'])*100/qc_countsdict['cx']}
            Dat.loc[exec_count] = DataRow
            exec_count+=1
            

            

KeyboardInterrupt: 

In [57]:
Dat

,Original Circuit,Original Circuit QASM,Qubit Count,Original Gate Count,Original Depth,H_0,S_0,CX_0,Optimized Circuit,Optimized Circuit QASM,...,Optimized Depth,H_Opt,S_Opt,CX_Opt,Time,% Gate Count Reduction,% Depth Reduction,% H Reduction,% S Reduction,% CX Reduction
0,"((Instruction(name='h', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,5,1,3,1,"((Instruction(name='h', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,5,1,3,1,0.384536,0.0,0.000000,0.000000,0.0,0.000000
1,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,5,1,1,3,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,5,1,1,3,0.050339,0.0,0.000000,0.000000,0.0,0.000000
2,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,5,2,2,1,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,3,0,2,1,0.163913,40.0,40.000000,100.000000,0.0,0.000000
3,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,5,2,1,2,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,1,0,1,0,0.188760,80.0,80.000000,100.000000,0.0,100.000000
4,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,5,2,2,1,"((Instruction(name='h', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,4,2,2,1,0.052909,0.0,20.000000,0.000000,0.0,0.000000
5,"((Instruction(name='cx', num_qubits=2, num_clb...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,4,2,1,2,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,3,0,1,2,0.179784,40.0,25.000000,100.000000,0.0,0.000000
6,"((Instruction(name='h', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,4,3,1,1,"((Instruction(name='h', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,3,1,1,1,0.204016,40.0,25.000000,66.666667,0.0,0.000000
7,"((Instruction(name='h', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,3,4,0,1,"((Instruction(name='cx', num_qubits=2, num_clb...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,1,0,0,1,0.050678,80.0,66.666667,100.000000,0.0,0.000000
8,"((Instruction(name='cx', num_qubits=2, num_clb...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,5,0,1,4,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,1,0,1,0,0.174587,80.0,80.000000,0.000000,0.0,100.000000
9,"((Instruction(name='s', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",2,5,4,2,2,1,"((Instruction(name='h', num_qubits=1, num_clbi...","OPENQASM 2.0;\ninclude ""qelib1.inc"";\nqreg q[2...",...,4,2,2,1,0.046956,0.0,0.000000,0.000000,0.0,0.000000


In [64]:
Dat.loc[18]

Original Circuit          ((Instruction(name='s', num_qubits=1, num_clbi...
Original Circuit QASM     OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2...
Qubit Count                                                               2
Original Gate Count                                                       5
Original Depth                                                            4
H_0                                                                       2
S_0                                                                       3
CX_0                                                                      0
Optimized Circuit         ((Instruction(name='h', num_qubits=1, num_clbi...
Optimized Circuit QASM    OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2...
Optimized Gate Count                                                      5
Optimized Depth                                                           4
H_Opt                                                                     3
S_Opt       

In [65]:
Dat.loc[18]['Original Circuit'].draw() 

┌───┐┌───┐┌───┐┌───┐
q_0: ┤ S ├┤ S ├┤ S ├┤ H ├
     ├───┤└───┘└───┘└───┘
q_1: ┤ H ├───────────────
     └───┘

In [66]:
Dat.loc[18]['Optimized Circuit'].draw() 

┌───┐┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ S ├┤ H ├┤ S ├
     ├───┤└───┘└───┘└───┘
q_1: ┤ H ├───────────────
     └───┘

In [68]:
Clifford(Dat.loc[18]['Original Circuit']) == Clifford(Dat.loc[18]['Optimized Circuit'])

True

In [69]:
Dat.loc[18]

Original Circuit          ((Instruction(name='s', num_qubits=1, num_clbi...
Original Circuit QASM     OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2...
Qubit Count                                                               2
Original Gate Count                                                       5
Original Depth                                                            4
H_0                                                                       2
S_0                                                                       3
CX_0                                                                      0
Optimized Circuit         ((Instruction(name='h', num_qubits=1, num_clbi...
Optimized Circuit QASM    OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2...
Optimized Gate Count                                                      5
Optimized Depth                                                           4
H_Opt                                                                     3
S_Opt       

In [50]:
Dat.to_csv('out.csv')

In [129]:
qc_test = QuantumCircuit(2)

#cP cP cP2 cHcpH 

qc_test.h(0)
qc_test.s(1)
qc_test.cx(1,0)
qc_test.h(0)
qc_test.cx(1,0)

qc_test.s(0)
qc_test.s(0) 
qc_test.cx(1,0)

qc_test.s(0) 
qc_test.cx(1,0)

qc_test.s(0) 
qc_test.cx(1,0)


qc_test.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤ H ├┤ X ├┤ S ├┤ S ├┤ X ├┤ S ├┤ X ├┤ S ├┤ X ├
     ├───┤└─┬─┘└───┘└─┬─┘└───┘└───┘└─┬─┘└───┘└─┬─┘└───┘└─┬─┘
q_1: ┤ S ├──■─────────■──────────────■─────────■─────────■──
     └───┘

In [130]:
Clifford(qc_test)

Clifford(array([[ True, False, False, False, False],
       [False,  True, False, False, False],
       [False, False,  True, False, False],
       [False, False, False,  True, False]]))

In [131]:
opt_subroutine(qc_test).draw()

q_0: 
     
q_1:

In [127]:
qc_ = QuantumCircuit(2)

qc_.h(0)
qc_.cx(1,0)
qc_.h(0)
qc_.cx(1,0)
qc_.h(0)
qc_.cx(1,0)
qc_.h(0)
qc_.cx(1,0)

qc_.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
     └───┘└─┬─┘└───┘└─┬─┘└───┘└─┬─┘└───┘└─┬─┘
q_1: ───────■─────────■─────────■─────────■──

In [128]:
opt_subroutine(qc_).draw()

q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ S ├┤ S ├
     └───┘└───┘

In [137]:
str(qc_).split('\n')[2]


'     └───┘└─┬─┘└───┘└─┬─┘└───┘└─┬─┘└───┘└─┬─┘'

In [138]:
tc1 = QuantumCircuit(2)
tc1.h([0,1])
tc1.cx(0,1)
tc1.h([0,1])

tc2 = QuantumCircuit(2)
tc2.s(0)
tc2.s(0)
tc2.s(0)

tc3 = QuantumCircuit(2)
tc3.h(0)
tc3.s(1)
tc3.cx(1,0)
tc3.h(0)
tc3.cx(1,0)
tc3.s(0)
tc3.s(0) 
tc3.cx(1,0)
tc3.s(0) 
tc3.cx(1,0)
tc3.s(0) 
tc3.cx(1,0)

tc4 = QuantumCircuit(2)
tc4.h(0)
tc4.cx(1,0)
tc4.h(0)
tc4.cx(1,0)
tc4.h(0)
tc4.cx(1,0)
tc4.h(0)
tc4.cx(1,0)

test_cases = {'1':tc1,'2':tc2,'3':tc3,'4':tc4}

In [139]:
for i in test_cases:
    print(i)

1
2
3
4
